# Game_life

Juan Manuel Velásquez Cadavid, 2208056.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import sys
import os
from os import listdir

En primera instancia, se calculan los estados de cada celda de la matriz original. Para ello, se traslada la matriz original en las direcciones de las celdas vecinas a través de la función np.roll, y de acuerdo a la diferencia entre la matriz trasladada y la original se asignará el nuevo valor a cada celda.

In [ ]:
def matrices(table, dim):
    """
    En esta función se definen las matrices principales:
    roll_u := traslada la matriz original hacia arriba,
    roll_d := traslada la matriz original hacia abajo,
    roll_l := traslada la matriz original hacia la izquierda,
    roll_r := traslada la matriz original hacia la derecha.
    roll_u_l := traslada la matriz original hacia arriba a la izquierda,
    roll_u_r := traslada la matriz original hacia arriba a la derecha,
    roll_d_l := traslada la matriz original hacia abajo a izquierda,
    roll_d_r := traslada la matriz original hacia abajo la derecha.
    """
    roll_u = np.roll(table, -1,axis=0)
    roll_d = np.roll(table,  1,axis=0)
    roll_l = np.roll(table, -1,axis=1)
    roll_r = np.roll(table,  1,axis=1)

    u = roll_u - table 
    d = roll_d - table
    l = roll_l - table
    r = roll_r - table

    roll_u_l = np.roll(roll_u, -1,axis=1)
    roll_u_r = np.roll(roll_u,  1,axis=1)
    roll_d_l = np.roll(roll_d, -1,axis=1)
    roll_d_r = np.roll(roll_d,  1,axis=1)

    u_l = roll_u_l - table
    u_r = roll_u_r - table
    d_l = roll_d_l - table
    d_r = roll_d_r - table
    
    return u, d, l, r, u_l, u_r, d_l, d_r

Para la evolución del tablero se tienen en cuenta las condiciones de mortalidad y natalidad. Se inicializan los valores requeridos para el conteo de dichas condiciones, así como un arreglo del número de celdas vivas (número de unos en la matriz), otro con el número de nacidos (nuevas celdas que satisfagan la condición y cambien su estado de 0 a 1), uno con el número de celdas que murieron, y otro que ocupe el paso de cada generación.

In [ ]:
def evolve(dim, gen, table, u,d, l, r, u_l, u_r, d_l, d_r):
    """
    Evoluciona el tablero original de acuerdo a las condiciones:
    1) Si una celda tiene dos o tres vecinas, vive. De lo contrario muere por soledad o sobrepoblación.
    2) Una célula "muerta" (apagada) nace si tiene exactamente tres vecinas vivas.
    """
    vecino  = 0
    nacer   = 0
    nacido_ = 0
    muerto_ = 0
    g = 1
    
    ones = np.count_nonzero(table == 1)   
    print('la población inicial es: ', ones) 
    pop = [ones]
    generation = [0]
    nacido = [0]
    muerto = [0]
    #remove()
    save_graph(gen, generation, pop, nacido, muerto)
    
    table_sig = table
    
    while(g <= gen):
        for i in range (1,dim-1):
            for j in range (1,dim-1):

                if(table[i,j]==1):

                    if(u[i,j]==0): vecino += 1
                    if(d[i,j]==0): vecino += 1
                    if(l[i,j]==0): vecino += 1
                    if(r[i,j]==0): vecino += 1
                    if(u_l[i,j]==0): vecino += 1
                    if(u_r[i,j]==0): vecino += 1
                    if(d_l[i,j]==0): vecino += 1
                    if(d_r[i,j]==0): vecino += 1

                    if(vecino <= 1 or vecino > 3):
                        table_sig[i,j] = 0
                        muerto_ += 1
                    vecino = 0

                else:

                    if(u[i,j]==1): nacer += 1
                    if(d[i,j]==1): nacer += 1
                    if(l[i,j]==1): nacer += 1
                    if(r[i,j]==1): nacer += 1
                    if(u_l[i,j]==1): nacer += 1
                    if(u_r[i,j]==1): nacer += 1
                    if(d_l[i,j]==1): nacer += 1
                    if(d_r[i,j]==1): nacer += 1

                    if(nacer == 3):
                        table_sig[i,j] = 1
                        nacido_ += 1
                    nacer  = 0
        
        nacido.append(nacido_)
        muerto.append(muerto_)
        nacido_ = 0
        muerto_ = 0
        
        table = table_sig
        save_table(g,table)
        
        ones = np.count_nonzero(table == 1)
        
        pop.append(ones)
        generation.append(g+1)
        
        u,d, l, r, u_l, u_r, d_l, d_r = matrices(table, dim)
        
        g += 1
    
    tab_fin = table.copy()
    save_graph(gen, generation, pop,nacido,muerto)

El resultado de cada ciclo se grafica y se salva.

In [ ]:
def save_table(g,table):
    """
    Salva las imagenes de los tableros en cada generación.
    """
    rc('font',**{'family':'serif','serif':['Times']})
    rc('text', usetex=True)
    name = str(g)
    plt.figure(figsize=(10,10))
    plt.title('Generación = '+name, fontsize=25)
    plt.axis('off')
    plt.imshow(table, cmap='gray')
    plt.savefig(name,cmap='gray')    
    plt.close()

In [ ]:
 def save_graph(gen, generation, pop, nacido, muerto):
    """
    Salva una gráfica con los datos de la población viva, los muertos y los nacidos en 
    función del la generación.
    """
    rc('font',**{'family':'serif','serif':['Times']})
    rc('text', usetex=True)
    plt.figure(figsize=(10,10))
    plt.plot(generation, pop, "b", label = r"Vivos")
    plt.plot(generation, nacido, "g", label = r"Nacidos")
    plt.plot(generation, muerto, "r", label = r"Muertos")
    plt.xlabel(r"Generación", fontsize = 25)
    plt.ylabel(r"Población", fontsize = 25)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.xlim(0, gen)
    plt.ylim(0, )
    plt.legend()
    plt.legend(fontsize=15)
    plt.grid()
    plt.savefig('population',cmap='gray')
    plt.close()

Cada que se ejecuta el código se eliminan todos los archivos con extención ".png". Para esto es necesario cambiar el directorio en el cual se estén guardando las imágenes.

In [ ]:
def remove():
    
    directory=os.listdir("/home/jm/Documentos/Cosas de la U/maestría/Modelado_1/Game_life_entregable")

    for item in directory:
        if item.endswith(".png"):
            os.remove(item)

Como dato inicial se genera una matriz de ceros y unos aleatorios de acuerdo a la semilla elegida, en la cual se eliminan los bordes (siempre en estado cero).

In [ ]:
def main(dim, gen, seed):
    """
    Aquí se define la matriz original con un arreglo aleatorio de dimensión "dim" ingresada por el
    usuario, cuya aleatoriedad está definida por la semilla "seed", de tal forma que sea posible generar
    una matriz aleatoria nueva cada que se varía este parámetro. El número de evoluciones es determinado 
    por el usuario.
    """    
    remove()
    np.random.seed(seed)
    table = np.random.randint(low=0, high= 2, size = dim*dim).reshape(dim, dim)
    table[0::1, 0::dim-1] = 0
    table[0::dim-1, 0::1] = 0
    
    save_table(0,table)
    u,d, l, r, u_l, u_r, d_l, d_r = matrices(table, dim)
    evolve(dim, gen, table, u,d, l, r, u_l, u_r, d_l, d_r)
    
    return

In [ ]:
main(dim=100, gen=100, seed = 1)